In [4]:
details={
    "Mortgage": {
    "SettleDate": "2007-12-15",
    "MaturityDate": "2020-12-15",
    "IssueDate": "2000-12-15",
    "OriginalBalance": 10000000,
    "RemainingBalance": 7500000,
    "GrossCouponRate": 0.0475,
    "NetCouponRate": 0.045,
    "ServicingFee": 0.0025,
    "PaymentFrequency": 12,
    "DayCountBasis": 4,
    "PaymentDelayDays": 14,
    "PrepaymentModel": {
      "Type": "PSA",
      "PSA_Speed": 100,
      "CPR_Function": "CPR = min(0.06, 0.002 * month)",
      "SMM_Calculation": "SMM = 1 - (1 - CPR)^(1/12)"
    },
    "DiscountCurve": {
      "Type": "OIS",
      "Currency": "USD",
      "AsOfDate": "2007-12-15",
      "Source": "Fed/OIS market data"
    },
    "RiskMeasures": {
      "DeltaEVE": True,
      "DeltaNII": True,
      "EffectiveDuration": True,
      "GapRisk": True,
      "BasisRisk": True
    },
    "FTP_Assumptions": {
      "Curve": "Bank Funding Curve",
      "Spread": 0.005
    },
    "Scenario": {
      "HorizonMonths": 12,
      "RateShock": "+200bp parallel",
      "BaseCurveSameAsShockCurve": True
    }
  }
}


In [5]:
# Generating mortgage cashflows (payments + prepayments) with strict day-count conventions.
# This code explains each step (prints brief explanations) and produces a per-period cashflow table.
# It assumes monthly payments (Period=12) and uses a PSA prepayment ramp with burnout.
# Adjust inputs below as needed.

from datetime import datetime, date, timedelta
from dataclasses import dataclass
from typing import List
import pandas as pd
import math

# Optional display helper available in this environment
try:
    from caas_jupyter_tools import display_dataframe_to_user
except Exception:
    display_dataframe_to_user = None

# ---------- Inputs (from your example + reasonable defaults) ----------
Settle = datetime(2007, 12, 15).date()
Maturity = datetime(2020, 12, 15).date()
IssueDate = datetime(2000, 12, 15).date()
GrossRate = 0.0475   # not used directly here, kept for completeness
CouponRate = 0.045   # coupon paid to investors (annual)
Delay = 14           # days payment is delayed (for settlement/accrual reporting)
Period = 12          # payments per year (monthly)
Basis = 4            # day-count basis code (we'll treat 4 as 30/360 PSA -> 30/360)
OriginalBalance = 10_000_000.0  # assumed original pool / loan size (you can change)
RemainingBalance = 10_000_000.0  # starting outstanding balance at Settle (for simplicity)

# PSA / prepayment settings
PSA_speed = 100               # 100% PSA (standard)
burnout_exponent = 0.5        # simple burnout: CPR_scaled *= (balance/original_balance)^burnout_exponent
cpr_cap = 0.60                # maximum allowed CPR (60% pa, extreme)
max_months_for_psa = 30       # months until CPR hits PSA cap (30 -> 6% at 100% PSA)

# ---------- Utility functions ----------

def add_months(d: date, months: int) -> date:
    y = d.year + (d.month - 1 + months) // 12
    m = (d.month - 1 + months) % 12 + 1
    # handle end-of-month semantics: clamp day to month's max day
    import calendar
    last_day = calendar.monthrange(y, m)[1]
    day = min(d.day, last_day)
    return date(y, m, day)

def months_between(start: date, end: date) -> int:
    return (end.year - start.year) * 12 + (end.month - start.month) + (1 if end.day >= start.day else 0) - 1

def year_fraction(start: date, end: date, basis: int = 4) -> float:
    # strict implementations for common bases. basis mapping:
    # 0: ACT/ACT (use actual/365.0 for simplicity), 1: 30/360 US, 2: ACT/360, 3: ACT/365, 4: 30/360 (PSA -> use 30/360 US)
    if start >= end:
        return 0.0
    days = (end - start).days
    if basis == 0:
        # Actual/Actual (approximate using 365 for simplicity)
        # For strict ISDA, you would account for year boundaries — here use ACT/365
        return days / 365.0
    if basis == 2:
        return days / 360.0
    if basis == 3:
        return days / 365.0
    # basis == 1 or 4 -> 30/360 US (NASD)
    # 30/360 US adjustments:
    d1 = start.day
    d2 = end.day
    m1 = start.month
    m2 = end.month
    y1 = start.year
    y2 = end.year
    if d1 == 31:
        d1 = 30
    if d2 == 31 and d1 == 30:
        d2 = 30
    days360 = (360 * (y2 - y1) + 30 * (m2 - m1) + (d2 - d1))
    return days360 / 360.0

# ---------- Build the payment schedule (monthly) ----------
# First payment date: next monthly cycle after Settle
first_payment = add_months(Settle, 1)
payment_dates = []
d = first_payment
while d <= Maturity:
    payment_dates.append(d)
    d = add_months(d, 1)

num_periods = len(payment_dates)

print("Generating cashflows:")
print(f" Settle date: {Settle}, Issue date: {IssueDate}, Maturity: {Maturity}")
print(f" Number of scheduled monthly payments: {num_periods}")
print(f" Day-count basis code: {Basis} -> using {'30/360' if Basis in (1,4) else 'ACT/365/360 variant'}")
print("")

# ---------- Compute level payment (annuity) using periodic rate = CouponRate / Period ----------
periodic_rate = CouponRate / Period
n = num_periods
if periodic_rate == 0:
    annuity = RemainingBalance / n
else:
    annuity = periodic_rate * RemainingBalance / (1 - (1 + periodic_rate) ** (-n))

print("Payment calculation:")
print(f" Periodic rate (monthly) = CouponRate / Period = {periodic_rate:.10f}")
print(f" Number of periods = {n}")
print(f" Level (scheduled) payment per period = {annuity:,.2f}")
print("")

# ---------- Loop and generate cashflows applying PSA prepayment and burnout ----------
rows = []
balance = RemainingBalance
prev_date = Settle  # accrual starts from settlement (or last payment date)
cumulative_prepay = 0.0

for idx, pay_date in enumerate(payment_dates, start=1):
    # accrual
    accrual_start = prev_date
    accrual_end = pay_date
    year_frac = year_fraction(accrual_start, accrual_end, Basis)
    
    # interest accrual for the period (strict daycount applied)
    interest = balance * CouponRate * year_frac
    
    # scheduled principal (annuity minus interest)
    scheduled_principal = annuity - interest
    if scheduled_principal < 0:
        scheduled_principal = 0.0
    
    # PSA-based CPR: CPR_m = min(0.06, 0.002 * month_since_issue)
    # month index for PSA is months since issue to accrual_end
    months_since_issue = months_between(IssueDate, accrual_end) + 1  # 1-based month count
    base_cpr = min(0.06, 0.002 * months_since_issue)  # 0.002 * m gives 0.2% * m until 6% at month 30 (100% PSA)
    
    # burnout scaling: reduce CPR as balance declines
    burnout_factor = (balance / OriginalBalance) ** burnout_exponent if OriginalBalance > 0 else 1.0
    adjusted_cpr = base_cpr * burnout_factor
    adjusted_cpr = min(adjusted_cpr, cpr_cap)
    
    # convert annual CPR -> monthly SMM
    smm = 1 - (1 - adjusted_cpr) ** (1 / 12.0)
    
    # prepayment base (amount eligible to prepay after scheduled principal)
    prepayment_base = max(balance - scheduled_principal, 0.0)
    prepayment_amt = prepayment_base * smm
    
    # ensure we don't prepay more than remaining principal
    total_principal = scheduled_principal + prepayment_amt
    if total_principal > balance:
        # final period: adjust scheduled principal so ending balance is zero
        total_principal = balance
        scheduled_principal = max(0.0, balance - prepayment_amt)
        prepayment_amt = max(0.0, balance - scheduled_principal)
    
    end_balance = balance - total_principal
    total_cashflow = interest + total_principal
    
    rows.append({
        "period": idx,
        "pay_date": pay_date,
        "accrual_start": accrual_start,
        "accrual_end": accrual_end,
        "year_frac": round(year_frac, 8),
        "begin_balance": round(balance, 2),
        "scheduled_payment": round(annuity, 2),
        "interest": round(interest, 2),
        "scheduled_principal": round(scheduled_principal, 2),
        "prepayment": round(prepayment_amt, 2),
        "total_principal": round(total_principal, 2),
        "end_balance": round(end_balance, 2),
        "total_cashflow": round(total_cashflow, 2),
        "base_cpr": round(base_cpr, 6),
        "burnout_factor": round(burnout_factor, 6),
        "adjusted_cpr": round(adjusted_cpr, 6),
        "smm": round(smm, 8),
        "months_since_issue": months_since_issue
    })
    
    # prepare for next period
    prev_date = accrual_end
    balance = end_balance
    cumulative_prepay += prepayment_amt
    if balance <= 1e-6:
        # loan fully repaid
        break

# ---------- Present results as a dataframe ----------
df = pd.DataFrame(rows)

# show first 36 rows for readability (or full if shorter)
if display_dataframe_to_user is not None:
    display_dataframe_to_user("Mortgage Cashflow Schedule (with PSA prepayment + burnout)", df)
else:
    # fallback printing
    pd.set_option('display.max_rows', 60)
    pd.set_option('display.float_format', '{:,.2f}'.format)
    print(df.head(36))

# End of script. The DataFrame 'df' contains the full cashflow schedule.



Generating cashflows:
 Settle date: 2007-12-15, Issue date: 2000-12-15, Maturity: 2020-12-15
 Number of scheduled monthly payments: 156
 Day-count basis code: 4 -> using 30/360

Payment calculation:
 Periodic rate (monthly) = CouponRate / Period = 0.0037500000
 Number of periods = 156
 Level (scheduled) payment per period = 84,787.10

    period    pay_date accrual_start accrual_end  year_frac  begin_balance  \
0        1  2008-01-15    2007-12-15  2008-01-15       0.08  10,000,000.00   
1        2  2008-02-15    2008-01-15  2008-02-15       0.08   9,901,525.97   
2        3  2008-03-15    2008-02-15  2008-03-15       0.08   9,803,448.35   
3        4  2008-04-15    2008-03-15  2008-04-15       0.08   9,705,761.62   
4        5  2008-05-15    2008-04-15  2008-05-15       0.08   9,608,460.27   
5        6  2008-06-15    2008-05-15  2008-06-15       0.08   9,511,538.88   
6        7  2008-07-15    2008-06-15  2008-07-15       0.08   9,414,992.06   
7        8  2008-08-15    2008-07-15  2